In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install mysql-connector-python sqlalchemy pymysql
import mysql.connector

conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="amit",
    database="project_360"
)

cursor = conn.cursor()
cursor.execute("SELECT * FROM customer_master LIMIT 5;")

for row in cursor.fetchall():
    print(row)

cursor.close()
conn.close()


('CustomerID', 'CustomerName', 'City', 'State')
('C00001', 'Customer_1', 'Pune', 'Delhi')
('C00002', 'Customer_2', 'Bangalore', 'Maharashtra')
('C00003', 'Customer_3', 'Delhi', 'Delhi')
('C00004', 'Customer_4', 'Hyderabad', 'Tamil Nadu')


In [2]:
customers = pd.read_csv("Customer_Master.csv")   # CustomerID, CustomerName, City, State
orders    = pd.read_csv("Orders.csv")           # OrderID, ProductID, CustomerID, StoreID, Quantity, GMV, Revenue
products  = pd.read_csv("Product_Master.csv")   # ProductID, ProductName, Category, Brand, Price
returns   = pd.read_csv("Returns.csv")          # ReturnID, OrderID, Reason
stores    = pd.read_csv("Store_Master.csv")

In [3]:
for df in [customers, orders, products, returns, stores]:
    df.drop_duplicates(inplace=True)

In [4]:
customers.fillna({"CustomerName": "Unknown", "City": "Unknown", "State": "Unknown"}, inplace=True)
products.fillna({"ProductName": "Unknown", "Category": "Unknown", "Brand": "Unknown", "Price": 0}, inplace=True)
stores.fillna({"StoreName": "Unknown", "City": "Unknown"}, inplace=True)

orders["Quantity"] = orders["Quantity"].fillna(0)
orders["GMV"]      = orders["GMV"].fillna(0)
orders["Revenue"]  = orders["Revenue"].fillna(0)

In [5]:
if "OrderDate" in orders.columns:
    orders["OrderDate"] = pd.to_datetime(orders["OrderDate"], errors="coerce")
else:
    orders["OrderDate"] = pd.date_range(start="2024-01-01", periods=len(orders), freq="D")

orders["OrderDate"] = pd.to_datetime(orders["OrderDate"], errors="coerce")

# Create time features

In [6]:
orders["Year"]  = orders["OrderDate"].dt.year
orders["Month"] = orders["OrderDate"].dt.to_period("M").astype(str)
orders["Week"]  = orders["OrderDate"].dt.to_period("W").astype(str)
orders["Date"]  = orders["OrderDate"].dt.date

# 3. MERGE DATASETS → MASTER SALES TABLE

In [7]:
sales = orders.merge(products, on="ProductID", how="left")
sales = sales.merge(customers, on="CustomerID", how="left")
sales = sales.merge(stores, on="StoreID", how="left")

In [8]:
# Merge returns by OrderID (no return quantity, only flag)
sales = sales.merge(returns[["OrderID", "ReturnID", "Reason"]], on="OrderID", how="left")
sales["is_returned"] = np.where(sales["ReturnID"].notna(), 1, 0)

In [9]:
# GMV already provided in Orders
sales["Net_Sales"] = sales["Revenue"]

In [10]:
# Assume Price is unit cost for now (no cost column given)
sales["Cost"]   = sales["Quantity"] * sales["Price"]
sales["Margin"] = sales["Net_Sales"] - sales["Cost"]
sales["Margin_%"] = np.where(
    sales["Net_Sales"] != 0,
    sales["Margin"] / sales["Net_Sales"] * 100,
    0
)

In [3]:
# 1. CLEAN CUSTOMER MASTER
# -----------------------------

customer.columns = customer.columns.str.strip()

# Fix inconsistent text
customer['City'] = customer['City'].str.title().str.strip()
customer['State'] = customer['State'].str.title().str.strip()

# Remove city/state mismatch manually if needed
city_state_map = {
    "Delhi": "Delhi",
    "Pune": "Maharashtra",
    "Hyderabad": "Telangana",
    "Bangalore": "Karnataka",
}

customer['State'] = customer['City'].map(city_state_map)

# -----------------------------
# 2. CLEAN PRODUCT MASTER
# -----------------------------

product.columns = product.columns.str.strip()

product['Category'] = product['Category'].str.title().str.strip()
product['Brand'] = product['Brand'].str.title().str.strip()

# -----------------------------
# 3. CLEAN STORE MASTER
# -----------------------------

store.columns = store.columns.str.strip()
store['City'] = store['City'].str.title().str.strip()

# -----------------------------
# 4. CLEAN ORDERS TABLE
# -----------------------------

orders.columns = orders.columns.str.strip()

# Convert numeric columns
orders['GMV'] = pd.to_numeric(orders['GMV'], errors='coerce')
orders['Revenue'] = pd.to_numeric(orders['Revenue'], errors='coerce')
orders['Quantity'] = pd.to_numeric(orders['Quantity'], errors='coerce')

# Remove negative or invalid values
orders = orders[(orders['Quantity'] > 0) & (orders['GMV'] > 0)]

# -----------------------------
# 5. CLEAN RETURNS TABLE
# -----------------------------

returns.columns = returns.columns.str.strip()

returns['Reason'] = returns['Reason'].str.title().str.strip()

# -----------------------------
# 6. VALIDATE FOREIGN KEYS
# -----------------------------

# Keep only valid Product IDs
orders = orders[orders['ProductID'].isin(product['ProductID'])]

# Keep only valid Customer IDs
orders = orders[orders['CustomerID'].isin(customer['CustomerID'])]

# Keep only valid Store IDs
orders = orders[orders['StoreID'].isin(store['StoreID'])]

# -----------------------------
# 7. MERGE ALL TABLES
# -----------------------------

merged = orders.merge(customer, on="CustomerID", how="left") \
               .merge(product, on="ProductID", how="left") \
               .merge(store, on="StoreID", how="left") \
               .merge(returns, on="OrderID", how="left")

merged.head()


,OrderID,ProductID,CustomerID,StoreID,Quantity,GMV,Revenue,CustomerName,City_x,State,ProductName,Category,Brand,Price,StoreName,City_y,ReturnID,Reason
0,O000001,P0288,C04812,S151,3,11204.69,9523.99,Customer_4812,Pune,Maharashtra,Product_288,Grocery,Brandb,3290.68,Store_151,Hyderabad,NaN,NaN
1,O000002,P0395,C04351,S141,4,453.03,385.08,Customer_4351,Bangalore,Karnataka,Product_395,Grocery,Branda,1837.94,Store_141,Bangalore,NaN,NaN
2,O000003,P0339,C02534,S183,1,1126.15,957.22,Customer_2534,Pune,Maharashtra,Product_339,Home,Brandb,4067.77,Store_183,Delhi,R00601,Quality Issue
3,O000004,P0358,C00476,S076,3,4819.73,4096.77,Customer_476,Chennai,NaN,Product_358,Grocery,Brandb,2206.56,Store_76,Bangalore,NaN,NaN
4,O000005,P0388,C01148,S024,1,4766.82,4051.80,Customer_1148,Delhi,Delhi,Product_388,Fashion,Brandd,1590.73,Store_24,Delhi,NaN,NaN
